### Submission guidelines

1. Fill in your name in the notebook in the top cell.
2. Fill in the gaps in the code where indicated. <br> Make sure that you:<br> - fill in any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE" <br> - do **not leave any** `raise NotImplementedError()` in the code
3. Do **NOT change the variable names**! However, you can add comments in the code.
4. Do **NOT remove any of the cells** of the notebook!
5. Discussion is allowed, but every student needs to hand a personal version of the lab. Plagiarism will be sanctioned!   
6. Before submitting, restart your kernel & **make sure that every cell runs**.<br>Code that doesn't run will not be scored.<br>The notebooks with all source code, and optional extra files need to be handed in using Ufora.<br> Make sure all your notebooks are already executed when you upload them (i.e. there should be output after the cells). 
7. **Zip** your lab assignment folder and name the archive: `Surname_Name.zip` <br> Keep the same folder structure as the provided lab assignment!<br> - <span style='color: red'>Do not rename any of the notebooks or files</span>!<br>



In [ ]:
NAME = "Cesar_Zapata"


Final tip: make sure you have answered every question and filled in all the required code by running through the notebook and searching for *YOUR ANSWER HERE* and *YOUR CODE HERE*!

Good luck!

---

# Lab: SQL


## Why SQL?

In small scale data exploration and analysis we use data from [flatfiles](https://www.wikiwand.com/en/Flat-file_database) (e.g., a few CSV files). This means you have loaded all data from the flatfiles into your computer's memory. This approach is OK in small scale settings where all data fits in your computer's memory.

For many purposes, loading *all* data from flatfiles into memory is not ideal, certainly not when the same queries are executed repeatedly. For example, every time you visit your Facebook homepage, *many* queries are executed against the huge amount of data Facebook has - you don't want to impose that first all data must be loaded into memory in such cases. You want your query to be executed fast, only on the data that matters to the query, and preferably you want that the results of popular queries are cached (remembered for faster retrieval). Data is therefore often stored in relational databases, and many applications execute queries on that data, using a querying language such as Structured Query Language (SQL).

To understand the basics of how relational databases structure the data differently than in flatfiles and why that allows for efficient querying, refer to this basic hypothetical flatfile example:

```
"Name",  "Age", "Height",  "Time", "Heart rate"
"Penny",  30,      182,     "8:00",     65
"Penny",  30,      182,     "9:00",     71
"Penny",  30,      182,    "10:00",     72
"Penny",  30,      182,    "11:00",     68
```

Observe that many redundant data is stored in the flatfile example above, i.e., the age and height for the same person.
A relational database will split the data over multiple concepts (so splitting the data over multiple tables) and model the relations between it.
The above data could have been modeled in a relational database with a `patients` table:

```
"Patient ID", "Name", "Age", "Height"
"1",          "Penny",  30,    182
```

and a `measurements` table, linked to the patients with the `Patient ID` column:

```
"Patient ID", "Time", "Heart rate"
"1",          "8:00",          65
"1",          "9:00",          71
"1",         "10:00",          72
"1",         "11:00",          68
```

Because of this structuring, relational databases allow to:
- **store data efficiently**: in the above example, the patient information is not repeated anymore for every measurement, as opposed to how it was done in the flatfile.
- **query data efficiently**: the querying engine must only load the data required for the query, e.g. only the records from the measurements table about the patient with ID 1. 
- **prevent data corruption**: correcting the age of Penny above only requires modifying one row in the patient table, whereas mistakes can easily happen in the flatfile, where all rows about Penny must be modified.
- **lock records in tables during a transaction** and commit modifications to the records if the transaction completed successfully or do a rollback in case the transaction must be cancelled for some reason.


## This lab

In this lab, you will practice writing SQL queries, in order to extract data from the MIMIC-III (v1.4) database. The extracted data will be used in a later lab, in which you will apply machine learning techniques.

This lab should take you maximum 5 hours to complete. If you detect that something is taking you many hours, endangering you to complete your lab in 5 hours, contact us.  
After submitting this notebook, automated tests will evaluate your queries.

### <a style='color:red'> Query limits</a>

This year we setup the database on a server (accessible through a connection string). To limit the (parallel) web traffic between our server and all your computers, we require that almost all queries are executed with a certain *limit*, e.g, `SOME QUERY ... LIMIT 5;` -> the *LIMIT 5* ensures that only the first 5 rows are returned.

In this notebook we added the required limits to each query. **Do NOT alter these limits as our automated tests will check them!**

### The MIMIC-III database

We set up the MIMIC-III database for you on a PostgreSQL server.
In order to set up the database we executed the required SQL instructions from [this repository](https://github.com/MIT-LCP/mimic-code).

After executing the SQL instructions [in this file](https://github.com/MIT-LCP/mimic-code/blob/master/buildmimic/postgres/postgres_create_tables_pg10.sql) all tables, their columns, and interrelations required to hold the MIMIC data were initialized.

#### First question: category of the above SQL queries

**Question**: Remember the theory class about SQL, or re-open the slides. Considering what the SQL instructions achieved above, what category of SQL queries do they belong to: DML or DDL? Motivate, in short, why.

**Answer**:
>

YOUR ANSWER HERE

# Introduction to database schema of MIMIC-III

For most applications, databases rapidly grow in number of tables and interrelationships. So, often a **Entity Relationship Diagram (ERD)** is made to quickly overview the database structure.

You can find an interactive ERD for MIMIC-III (`mimiciii`) at https://mit-lcp.github.io/mimic-schema-spy/relationships.html.

For the remainder of the lab, we will however focus on these 4 tables of the `mimiciii` database:

- patients
- icustays: admissions of patients to the intensive care unit (ICU).
- admissions: Hospital admissions associated with an ICU stay.
- services: Hospital services that patients were under during their hospital stay.

**Tip**: use this to check out the table's properties mentioned above https://mit-lcp.github.io/mimic-schema-spy/index.html

**Tip**: Given the limited number of tables we work with, the above ERD link will not be very helpful to you, *however* that website also provides you with **descriptions of what less intuitive column names mean**, such as `dod_ssn`: see https://mit-lcp.github.io/mimic-schema-spy/tables/patients.html. You'll need this further on in this lab for building queries.

Another option is checking the comments on tables and columns. The web page https://mit-lcp.github.io/mimic-schema-spy/tables/patients.html was actually constructed by reading these table and column properties.

# Executing SQL queries against the database

There are several ways to execute SQL queries:

- Using a **graphical interface** (`pgAdmin` for PostgreSQL)
- From the **command line, using the SQL Shell** (`psql` for PostgreSQL), this is particularly practical
  * for quickly trying out test queries in environments in which the graphical interface is unavailable, 
  * or when you want to create scripts that must execute a set of queries, such that these queries can be triggered without having to open the graphical interface.
- From **application code, using a library** (e.g. `sqlalchemy` or `psycopg2` for PostgreSQL)  

For this lab, we are going to query the MIMIC-III database from some code in this notebook, using the `sqlalchemy` library (uses `psycopg2` under the hood). As we host the database on a server, you will first need to connect to the database over the internet before you can start querying.

**Note**:
> if you work on linux make sure to install this package first `libpq-dev`, this is only relevant when you got an install error on the `psycopg2` package!

In [ ]:
# Install the requirements
!pip install psycopg2-binary pandas 'sqlalchemy<2.0'

In [ ]:
import sqlalchemy
import pandas as pd

In [ ]:
# Create the connection to the database 
user = "student"
password = "dahcc"
host = "<ASK TA's>"
port = 5432


conn = sqlalchemy.create_engine(
    f"postgresql://{user}:{password}@{host}:{port}/mimic",
    connect_args={"options": '-c statement_timeout=15s'}
)

In [ ]:
# Note how `mimiciii` represents the database name
# and `icustays` represents a table
query = """
SELECT *
FROM mimiciii.icustays as stays
JOIN mimiciii.patients p ON stays.subject_id = p.subject_id
LIMIT 5;
"""
pd.read_sql_query(query, conn)

The above code returns a connection, ready for you to use for sending queries to the database engine.

It's up to you from now on.

Use the following resources for help to complete your lab:
- about constructing SQL queries:    the theory slides
- about MIMIC's less interpretable column names:    https://mit-lcp.github.io/mimic-schema-spy/tables/patients.html
- about what is possible with SQL specifically on PostgreSQL database servers:    the PostgreSQL documentation (https://www.postgresql.org/docs/12/)
- about the querying library you are using:    the psycopg2 documentation (https://www.psycopg.org/docs/)

You'll probably only need the first two resources.

Let's test our first query.


In [ ]:
# This shoud return a  count of 46520
query = """
SELECT COUNT(*)
FROM mimiciii.patients;
"""
pd.read_sql_query(query, conn)

In [ ]:
# This shoud return the first 5 rows of the patients table
query = """
SELECT *
FROM mimiciii.patients
LIMIT 5;
"""
pd.read_sql_query(query, conn)

## Investigating ICU stays recorded in the MIMIC-III database

The goal of this lab is to show you how you would extract data from a database for machine learning purposes.

The goal of a machine learning project could be to investigate the mortality of people that stayed at the Intensive Care Unit (ICU).

The `icustays` table will thus be the main point of interest from which we will extract data.

> First, can you fill the query below such that the cell will print the **amount of stays at the ICU** that are recorded in this dataset?

> Hint: do not just print all patients to avoid unnecessarily burdening your database: **this query requires an aggregate row function**.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> It may be interesting to first get an idea of **what the `icustays` records look like**. Query only 5 rows from this table.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 5;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> **Curious to see what the record length of a patient's visit to the ICU is?** List all stays at the ICU, in descending length of stay. Limit the result set to only 10 items. Give the resulting column name the more intuitive name 'length_of_stay'.

Hints:

1. Not all column names are intuitive. Use https://mit-lcp.github.io/mimic-schema-spy/tables/icustays.html.
2. Notice something weird? A special value may pop up in the result, stating that the length of stay is not known. Note that it is printed with a different name here due to conversion to python datatypes, you need to know however what the equivalent name for that is in database terms to complete your query. Add a filter that will remove these values (database naming) for now, such that you get a proper answer to your question of what the record stay is.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 10;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

### Collecting basic features about the patients that stayed in the ICU

> All patient information is available in the patients table, while we have worked on the icustays table. Query only 5 records from the patients table to get an idea of what they look like.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 5;
"""
# YOUR CODE HERE
raise NotImplementedError()
df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> The features we are going to collect are, however, about the **ICU stays**. Query 5 rows about ICU stays below and add the patient info in the rows.

Hints: 

1. Think about how multiple tables can be combined, refer back to the theory slides if necessary. 
2. Check the printed rows above or the table overview website to see which column links both tables.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 5;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

**Question**: Look back to the column you used for linking both tables. Tell us below:

- What sort of key is used for this in the icustays table,
- What sort of key is used for this in the patients table.

**Answer**:
>

YOUR ANSWER HERE

> Many people end up in the hospital many times in their lives. Can you group the ICU stays by patient and count the number of rows? The result then is, of course, the number of ICU stays per patient. Give the count the alias 'number_of_stays', sort it in descending order and limit to 20 counts.  
> The output should contain 2 columns; `subject_id` and `number_of_stays`.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 20;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Some queries get complicated. You can often check your results in another way. Count, for your champion of ICU stays above, the number of stays in the icustays table. Does it match with the number of stays you have listed above? If not, revise your above query and verify your result again with the query below.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> The goal of the machine learning lab will be to predict mortality. Let's first just look at what the mortality rate actually is in the ICU. Find out how many people have a decease date between the intime and outtime (inclusive) of their icustay. Remember you can use https://mit-lcp.github.io/mimic-schema-spy/tables/patients.html, for example to find out which column holds a person's decease date. Use the "general" decease date column, not the one recorded by the hospital or from the social security records, as the "general" decease date column is filled in the most.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Now calculate the number of people that have been in the ICU. Remember: you cannot just count the number of records in the icustays table, since we detected some champions in staying in the ICU often. In other words: count the DIFFERENT people having stayed in the ICU.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Now, in order to calculate the mortality rate in the ICU, **create a nested query** that reuses your last two queries and essentially divides the result of the deaths count by the patient count and multiplies by 100 to get the percentage. Give the result the alias "mortality_rate_at_ICU". If your result is 0, it is likely because you performed an integer division. Convert the queried patient count *and* patient count to float.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Copy your death count query to the cell below. Can you modify it to group it by date and sort the death count in descending order? Limit your results to 15 daycounts.  
> The output should contain 2 columns; `dod` and `count`.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 15;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> One important feature about patients for predicting mortality will be the patient's age. List 20 icustays records with their corresponding age.

1. First print the age as = ICU entering time - patient birthdate
2. The result so far will be in number of days. You may see ages listed in hours, minutes and seconds instead for patients in the ICU, think about how that is possible (it certainly is). Anyway, the results as you see them are printed by the python framework like that, what the database returns is **number of days**. Modify your select statement to end up with the number of years. Check the documentation https://www.postgresql.org/docs/12/functions-datetime.html for a function to extract **epochs**, add casting to float and using simple mathematics on that.
3. Sort the ages in descending order. Unrealistic ages will be listed. The creators of the MIMIC database have replaced all ages > 89 with values of about 300, because the exact age would otherwise make deceased patients identifiable. Modify your SQL statement such that patients with an age > 89 all have the age of exactly 300 years; and to avoid having to repeat your age calculation statement, consider the query you had as an inner query (```(SELECT ... ) AS patient_ages```) and build your age modification query as an outer query around it.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 20;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Check your query's result: copy your query to the below cell, remove the 20 records limit and print `df_result.describe()` instead of just `df_result`.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result.describe()

In [ ]:
############################# READ ONLY #############################

The maximum patient age should now of course be 300 years. If not, check and improve your query.

## Extracting the diagnoses

> Query just 5 records from the `diagnoses_icd` table to have a first look at its records.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 5;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> The `icd9_code` is a code, internationally agreed upon, that tells which disease was diagnosed for a patient (determined by the `subject_id`) during one of his hospital admissions (determined by the `hadm_id`). Now, using those 2 keys, link the `icd9_code` to the icustays table records. Limit your results to 5 ICU stays for now. Beware: you'll need to link the tables using 2 keys now! Also: use a special type of join to make sure that ICU stays for which patients couldn't get a diagnosis are still in the results.  
> The output columns should be; `icd9_code, row_id, subject_id, hadm_id, icustay_id, dbsource, first_careunit, last_careunit, first_wardid, last_wardid, intime, outtime, los`

In [ ]:
query = """
#+++ your answer here +++
LIMIT 5;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Copy the query of above and add a filter to check *how many* ICU stays actually *didn't get a diagnosis*. That of course also means you need to remove the LIMIT from your query here.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Now, going back to the query that listed the ICD codes (you can copy it below): we're of course not familiar with these codes. So modify your query to add a join below with the `d_icd_diagnoses` table, such that the name of the diagnosed disease is shown for each record. Only select the columns `subject_id, hadm_id, icustay_id, intime, outtime, row_id, icd9_code, short_title, long_title` (in this exact order). Limit the results to 5 rows.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 5;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

## Extracting vital signs of the patients

You will aggregate vital sign measurements of patients during the first 24 hours after ICU admission, such that in the machine learning lab, you will have aggregate information about the patient's health that will (clearly) still show most information about their condition - before their treatment started working. 

Of course, maybe some patients will have been cured in less than 24 hours, but on the other hand we want to gather enough vital signs about patients that stayed in the ICU for a long time. So it's merely a trade-off and we just picked "a" timeframe after hospital admission that intuitively makes sense.

> Query 5 rows of the d_items table first to again get an idea of the content.

In [ ]:
query = """
#+++ your answer here +++
LIMIT 5;
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> Search for d_items that have a label that contain the word 'Heart'. There is no need for a record limit on your query.

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

> So there are many measurements about patient's heart condition. Let's keep it simple and focus on just 'Heart Rate'. Query just these records (again, there is no need for a limit).

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

The above should print 2 results. 

> Now aggregate heart rate for 1 patient:

1. start from the icustays table and join it with the chartevents
2. count all chartevents that are Heart Rate measurements (**itemid = 211 instead of on the label 'Heart Rate'**, that's faster to execute) about the patient with subject_id 13033, **within 1 day after the intime of the patient in the ICU**.
3. modify your query to group by the icustay_id, such that it will print the count of chartevents per ICU stay.
4. modify your select statement from chartevent counts to the following aggregates of chartevents' `valuenum`: minimum, maximum, average and standard deviation. You may want to check the postgreSQL documentation for the function names for these aggregations.

The output should have the following columns: `lowest_heart_rate, highest_heart_rate, heart_rate_avg, heart_rate_stdv` (in this exact order).

In [ ]:
query = """
#+++ your answer here +++
"""
# YOUR CODE HERE
raise NotImplementedError()

df_result = pd.read_sql_query(query, conn)
df_result

In [ ]:
############################# READ ONLY #############################

## BONUS: Final extraction of a full CSV file of readily usable features for machine learning purposes

> <div style='color:red'>
    This question is <b>bonus</b> and will not be graded, however, we highly encourage to atleast attempt this task!
</div>
    
It is now up to you to create one big query to combine all the information you extracted piece by piece above.
We help you assemble your bigger query:
1. Start with selecting from icustays (and name the columns like that!): hospital_admission_id, patient_id, first_icu_care_unit, last_icu_care_unit, first_icu_ward_id, last_icu_ward_id, icu_in_time, icu_out_time, length_of_stay_in_ICU
2. Join with the patients table and add the features patient_gender
3. Add the calculation of patient_age in the SELECT statement just like you have added it in the SELECT of a query you wrote earlier, then consider the entire query you had so far as a subquery, around which you write the outer query you made before to correct ages: 300 if older than 89 otherwise just the patient's age; in the SELECT of the outer query, select all columns from the inner query *and* the corrected age (patient_age_corrected).
3. Add the mortality labels: previously, we established that a patient died during the ICU stay if his date of death (dod) was between the intime and outtime of the icustay. Add this in a CASE WHEN condition for a new column in the inner query, called died_during_ICU_stay and make it True when the condition is met, else False. Verify by running the entire query that this mortality label is in the results.
4. Add the disease labels: in the inner query, add the joins with the icustays table with the diagnoses_icd and d_icd_diagnoses tables and select the dname.long_title as diagnosed_disease (still in the inner query). Verify by running the entire query that this disease label is in the results.
5. Join your **inner** query with the heart rate aggregation query you made above - give it an alias, e.g. patient_heart_agg, and join on the icustay_id. Modify your outer query's select statement to make it include the columns resulting from the heart rate aggregation subquery.
6. Change the last line in your cell to no longer make it print `df_result` but to write it to a CSV file: `df_result.to_csv(path_or_buf='icustays_features_and_labels.csv')`. Execute and already test if the CSV is generated.
7. Remove the limit on the number of icustays that you normally still have in your query and execute the query. It may take longer to execute due to the big amount of data to process.

**Note**: you may notice that multiple records are created for the same patient and ICU visit, due to the fact that at that time, the patient received multiple diagnoses. That is fine.

> As this takes a lot of server-time to execute the query, we ask you to only provide the query string in the markdown cell below, on which (non-graded) feedback will be given.

**Answer**:

```sql
-- your query here!
LIMIT 5;
```

YOUR ANSWER HERE


The CSV file you extracted contains a collection of interesting features about patient visits to the ICU that will be reused in the machine learning lab.

In that lab, you will predict things like diagnoses or mortality, but note that your predictions will only be as strong as the information that you extracted, which was mainly the patient's vital signs.
Many patients in a hospital are also subjected to blood and urine tests, about which there is *also* data in the MIMIC database. Also adding information about this to your extracted data will probably make your predictions better. But that falls out of the scope of this lab assignment though. This lab was long enough already ;-)

## Material You Need to Submit

For this lab session, the notebook containing all code snippets and query results (\*) needs to be handed in using Ufora. 

Upload your notebook before next week, i.e., 7/03/2023 11:59PM CET!

(\*) Make sure all your notebooks are handed in already executed (i.e. there should be output after the cells).